## CCNSS 2018 Module 5: Whole-Brain Dynamics and Cognition
# Tutorial 2: Introduction to Complex Network Analysis (II)


*Please execute the cell bellow in order to initialize the notebook environment*

In [2]:
!rm -rf data ccnss2018_students
!if [ ! -d data ]; then git clone https://github.com/ccnss/ccnss2018_students; \
                        cp -rf ccnss2018_students/module5/2_introduction_to_complex_network_analysis_2/data ./; \
                        cp ccnss2018_students/module5/net_tool.py ./; fi

Cloning into 'ccnss2018_students'...
remote: Counting objects: 923, done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 923 (delta 0), reused 3 (delta 0), pack-reused 918
Receiving objects: 100% (923/923), 64.81 MiB | 166.00 KiB/s, done.
Resolving deltas: 100% (278/278), done.


In [0]:
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import math                        # import basic math functions
import random                      # import basic random number generator functions
import csv                         # import CSV(Comma Separated Values) file reading and writing
import scipy as sp                 # import scipy
from scipy import sparse           # import sparse module from scipy
from scipy import signal           # import signal module from scipy
import os                          # import basic os functions
import time                        # import time to measure real time
import collections                 # import collections
import networkx as nx              # import networkx

import sys
sys.path.append('../')
import net_tool as net             # import net_tool, a network analysis toolbox from tutorial #1 

data_folder = 'data'

print('Available data files:\n'+'\n'.join(sorted(os.listdir(data_folder))))

data_file_1 = os.path.join(data_folder, 'george_baseline_44.txt')
data_file_2 = os.path.join(data_folder, 'george_propofol.txt')
data_file_3 = os.path.join(data_folder, 'george_ketamin.txt')
data_file_4 = os.path.join(data_folder, 'george_medetomidine.txt')

# Objectives

In this notebook we will construct a fuctional network from a given time series. Following up on the powerpoint tutorial, we will first construct a  functional network from the brain signals, and compare functional network properties for different states of the brain.


## Background

Network theory (graph theory) measures can be applied to any kind of network, including the brain. Structural networks of various species are good examples. We can also construct fuctional networks from time series data we observe using various techniques such as fMRI, EEG, ECoG, and MEG.

Using an ECoG data from a macaque as an example, We will go through the following steps:

* Appy a measure (PLI: phase lag index) to two time series, and construct a PLI matrix.
* Construct a network from the PLI matrix, by applying a threshold.
* Apply various network measures to the resulting network.
* Construct the functional networks for different brain states, and compare how they differ from each other.
* (Optional) Divide the time series into small time windows, and construct functional network for each time window.



The example we will analyze is a thirty second - segment of whole brain ECoG data of a macaque monkey named George, from an eyes closed resting state. The sampling freqeuncy is 1000 Hz, resulting in total of 30,000 time points for each channel. The data consists of signals coming from 106 areas that cover the left hemisphere. The data is preprocessed, by applying a band path filter to remove the alpha wave component (7-13 Hz) from the signal. Alpha waves are correlated with global interactions of the brain for many instances of the brain states.

In [0]:
george_base = [ row for row in csv.reader(open(data_file_1,'r'),delimiter='\t')]
george_base = np.array(george_base).astype(np.float32)

In [0]:
george_propofol = [ row for row in csv.reader(open(data_file_2,'r'),delimiter='\t')]
george_propofol = np.array(george_propofol).astype(np.float32)

**EXERCISE 0: Calculating* i)* the phases of oscillating signals, and* ii)* the differences between the phases from two signals. Read through and understand the code, which will be used in later exercises (Exercise #4).
**


$i)$ Every oscillating signal $S_j$ can be represented by its amplitude and its phase:

$$ S_j(t) = r_j(t) e^{i \theta_j(t) } = r_j(t) ( \cos \theta_j(t) + i \  \sin \theta_j(t) ) $$

Using this representation, we could assign $phase$ $\theta_j$ to the signal at every time point $t$. One way of computing the phase of a signal for each time point is using the ***Hilbert transform***. 

•  We can obtain the signal in the form of above representation by `sp.hilbert`($S_j$). After that, we could use   `np.angle()` to get the angle at each time point $t$: `np.angle(sp.hilbert`( $S_j$ ) `).` 

$$ $$

$ii)$ After getting the angle $\theta_j$ of each signal $S_j$, we can calculate the differences between phases:

$$ \Delta \theta_{jk}(t) = \theta_j(t) - \theta_k(t) \\ $$

Best way to calculate the phase difference, again is to calculate it in the exponent form:

$$  e^{i \Delta \theta_{jk} (t)} = e^{i ( \theta_j (t) -  \theta_k (t)  )   },\\ $$

then take the angle of $  e^{i \Delta \theta_{jk} (t)} $:

$$  \Delta \theta_{jk} (t) = arg ( e^{i \Delta \theta_{jk} (t)} ) .\\ $$

We can obtain the angle by using `np.angle()`.

This phase difference gives a valuable information about the "directionality" between pair of oscillators.

• Calculate the $\theta_{ij}$ between all pairs of time series, and build a phase-difference matrix. Each elements of the matrix containing time averaged phase difference $\langle \theta_{ij} \rangle _t$ between $i$ and $j$. The resulting matrix will be anti-symmetric.

• From the phase-difference matrix we constructed, compute the average phase-difference for each node. Calculate the row-sum of the matrix:

$$ \theta_i = \frac{1}{N} \sum_{j=1}^{N} \langle \theta_{ij} \rangle _t,$$

then we can have a vector of averaged phase-differences, each element of the vector corresponding for each node.

This average phase-difference for each node will tell us whether one node is phase-leading or phase-lagging with respect to other nodes over a given period of time.

In [0]:
# getting the phases from the signals, using np.angle and sp.signal.hilbert

george_base_angle = np.angle(sp.signal.hilbert( george_base,axis=0) )
print("size of george_base_angle is:" , george_base_angle.shape )

In [0]:
def phase_diff_mat(theta):
    
    # theta must has dimension TxN, where T is the length of time points and N is the number of nodes
    
    N_len = theta.shape[1]
        
    PDiff_mat= np.zeros((N_len,N_len))    
    for ch1 in range(N_len):
        for ch2 in range(ch1+1,N_len):
            PDiff=theta[:,ch1]-theta[:,ch2]                     # theta_ch1 - theta_ch2
            PDiff_exp_angle = np.angle( np.exp(1j*PDiff) )      # angle of exp (1i * (theta_ch1-theta_ch2)   )
            PDiff_exp_mean = np.mean(PDiff_exp_angle)           # mean of the angle with respect to time
            PDiff_mat[ch1,ch2] = PDiff_exp_mean                 # put the mean into the matrix
            PDiff_mat[ch2,ch1] = -1*PDiff_exp_mean              # the matrix will be anti-symmetric
            
    PDiff_mean = np.mean(PDiff_mat,axis=1)                      # calculate the mean for each node, with respect to all the other nodes
    
    #alternative code
    #arr = np.array([np.roll(theta, i, axis=1) for i in range(N_len)])
    #PDiff_mat = theta[None, :] - arr
    #PDiff_mean = PDiff_mat.mean(1)
    
    return PDiff_mean,PDiff_mat

**EXERCISE 1: Calculating the PLI for two given time series**

The data is in a form of 30,000x106 (# of time points  x  # of channels) sized matrix. We will measure $PLI$s between all possible pairs of channels.

We now define $dPLI$ (directed phase-lag index) as the following:

$$ dPLI_{ij} = \frac{1}{T}\sum_{t=1}^{T} sign ( \Delta \theta_{ij} (t)  )   \, $$

where

$$ \Delta \theta_{ij} = \theta_i - \theta_j ,$$

and 

$$ sign (  \theta_i - \theta_j  ) = 
\begin{cases}
1 & if  \  \Delta \theta_{ij} > 0 \\   
0 &  if \  \Delta \theta_{ij} = 0 \\
-1 &  if \  \Delta \theta_{ij} < 0. \\
\end{cases}  \\ $$

$dPLI$ will range from 1 to -1, and give us information about which signal is leading another. \

If we take absolute value of $dPLI$, we get $PLI$ (phase lag index):

$$\\ PLI_{ij} =|dPLI_{ij}| = | \langle  sign ( \Delta \theta_{ij} )  \rangle_t  | .\\$$


$PLI$ will range from 0 to 1, and give us information about whether two signals have consistent phase-lead/lag relationship with each other over given period of time.

•  Plot the time series for the first 3 channels of `george_base` (first 500 time points)

•  Plot the time series for the first 3 channels of `george_base_angle` (first 500 time points).

•  Compute $PLI_{ij}$ for all pairs of $i$ and $j$, and make $PLI$ matrix. The resulting matrix will be symmetric. You can use `np.sign()`.

In [0]:
# Write your code for plotting time series 

**EXPECTED OUTPUT**

![](https://github.com/ccnss/ccnss2018_students/raw/master/module5/2_introduction_to_complex_network_analysis_2/figures/1_ts_angle.png)

In [0]:
def cal_dPLI_PLI(theta):
 
    # insert your code for calculating dPLI and PLI
    # theta must has dimension TxN, where T is the length of time points and N is the number of nodes
    # outputs PLI matrix containing PLIs between all pairs of channels, and dPLI matrix containg dPLIs between all pairs of channels

  
    return PLI,dPLI

In [0]:
george_base_PLI, george_base_dPLI = cal_dPLI_PLI(george_base_angle)
print(george_base_dPLI[:5,:5])

**EXPECTED OUTPUT**
```
[[ 0.         -0.09446667  0.0348     -0.05666667  0.28      ]
 [ 0.09446667  0.          0.04926667  0.00693333  0.341     ]
 [-0.0348     -0.04926667  0.         -0.0614      0.2632    ]
 [ 0.05666667 -0.00693333  0.0614      0.          0.3316    ]
 [-0.28       -0.341      -0.2632     -0.3316      0.        ]]
 ```


**EXERCISE 2: Constructing network connectivity matrix**

We can construct a network from the above PLI matrix. Two approaches are possible. We can apply a threshold value for the PLI matrix and turn it into a binary network. Or, we can take the PLI value as is, and turn the matrix into a weighted network. We will take the first approach.

• Binary network approach: one must determine a right threshold value for the matrix. For example, you can choose a value such that highest 30% of the PLI values between nodes will turn into connection.

• (Optional) Weighted network approach: we can take the PLI value itself as the weighted link between two nodes.



In [0]:
def cal_mat_thresholded(data_mat, threshold):
    
    
    # insert your code here
    # input is the original matrix with threshold
    # output is the thresholded matrix. It would be symmetric.

    return data_mat_binary

In [0]:
threshold = 0.3
george_base_PLI_p3 = cal_mat_thresholded(george_base_PLI,threshold)
print("sum of george_base_PLI_p3:", np.sum(george_base_PLI_p3))

**EXPECTED OUTPUT**
```
sum of george_base_PLI_p3: 3372.0
```

**EXERCISE 3: Applying network measure to the functional network**

We now have a resulting functional network from a macaque ECoG data. Now we can apply network measures to this network.

• Apply network measures to this network, such as $C, L, E$ and $b$ (clustering coefficient, characteristic path length, efficiency, and betweenness centrality).

(If you prefer, you can use functions that we provide in net.py. Ask tutors for the details.)


In [0]:
# insert your code here

**EXPECTED OUTPUT**
```
C: 0.4405029623271814
E and L: 1.735130278526505 0.6451332734351602
b: 38.594339622641506
```

**EXERCISE 4: Computing phase measures for the functional network**

We can define a mean of $PLI_i$ over all other nodes as follows:

$$ PLI_i = \frac{1}{N-1} \sum_{j=1,\ j \neq i }^{N}  PLI_{ij} ,$$

This quantity will tell us how persistantly a node is locked with respect to other nodes, over a given period of time. Usually, the node with high $PLI_i$ is the one with high degree in a network: the $k_i$ and $PLI_i$ of a node $i$ is correlated.

We can also define a mean of $dPLI_i$ over all other nodes as follows:

$$ dPLI_i = \frac{1}{N-1} \sum_{j=1,\ j \neq i}^{N}  dPLI_{ij} ,$$

This quantity will tell us how persistantly a node is phase-leadaing or phase-lagging with respect to other nodes, over a given period of time. This quantity is correlated with the average phase-difference $\theta_i$ which we defined in earlier exercise.

• Do a scatterplot of the mean PLI and mean dPLI. Is there any pattern between these two quantities? Calculate the Pearson correlation coefficient between these two vectors.

• Also, you can do a scatterplot of degree of each node vs. average phase-difference. Do they resemble above the scatter plot?

In [0]:
# insert your code for calculating mean dPLI and PLI, mean phase, and degree of the network 

george_base_PLI_mean = 
george_base_dPLI_mean = 
george_base_phase_diff_mean,_ = phase_diff_mat(george_base_angle)
george_base_PLI_p3_degree = 

In [0]:
plt.figure()
for i in range(len(george_base_PLI_mean)):
    plt.plot(george_base_PLI_mean[i],george_base_dPLI_mean[i],'C0s')
    plt.text(george_base_PLI_mean[i],george_base_dPLI_mean[i],str(i))
    plt.xlabel('PLI')
    plt.ylabel('dPLI')
    plt.title('dPLI vs PLI')
plt.show()

corr_PLI_dPLI = np.corrcoef(george_base_PLI_mean,george_base_dPLI_mean)
print("corr. of PLI and dPLI is:", corr_PLI_dPLI[1,0])

plt.figure()
for i in range(len(george_base_PLI_p3_degree)):
    plt.plot(george_base_PLI_p3_degree[i] , george_base_phase_diff_mean[i],'C0s' )
    plt.text(george_base_PLI_p3_degree[i] , george_base_phase_diff_mean[i],str(i))
    plt.xlabel('k')
    plt.ylabel('theta')
    plt.title('theta vs k')
plt.show()

corr_degree_phase = np.corrcoef(george_base_PLI_p3_degree , george_base_phase_diff_mean)
print("corr. of degree and phase is:", corr_degree_phase[1,0])


**EXPECTED OUTPUT**

![](https://github.com/ccnss/ccnss2018_students/raw/master/module5/2_introduction_to_complex_network_analysis_2/figures/4_dpli_vs_pli.png)
```
corr. of PLI and dPLI is: -0.5848065158893657

```
![](https://github.com/ccnss/ccnss2018_students/raw/master/module5/2_introduction_to_complex_network_analysis_2/figures/4_theta_vs_k.png)
```
corr. of degree and phase is: -0.5082925792988023
```

**EXERCISE 5: Dividing the data into moving time windows (optional)**

Sometimes the time length of the data is large. Or, one wants to investigate the changes that occurs in finer time resolution. For example, we can apply a time window of 2 seconds with an overlap of 1 second to the data, dividing the data into 29 time segments of size 2000x106 matrix.



• Write a code for a function that divide a given time series into moving time windows.

• Using the codes from Exercise 1 and 2, construct a connectivity matrix for each time window.

• We can now apply network measures to the resulting connectivity matrices.

In [0]:
win_len = 2000
win_start = 10000
overlap = 1000

PLI_win = []
dPLI_win = []

for idx in range(0, george_base_angle.shape[0], overlap):
  
    temp = cal_dPLI_PLI(george_base_angle[idx:idx+win_len])
    PLI_win += [temp[0]]
    dPLI_win += [temp[1]]
    
PLI_win = np.array(PLI_win[:-1])
dPLI_win = np.array(dPLI_win[:-1])

**EXERCISE 6: Comparison between two different states of brain (optional, possible for mini projects)**

The above analysis can be repeated to different states of the brain. For example, we can construct the network from anesthesized unconcious states. The provided data is from anesthetized George, induced with propofol. We can construct the connectivity network and apply network measure.

• Repeat the processes in Exercise 1 and 2 to construct the resulting fuctional network.

• Apply network measures as in Exercise 3, and phase measures as in Exercise 4. Compare the result with the resting state network. How are they different from each other?


**EXERCISE 7: Phase coherence (optional, possible for mini projects)**

There are many measures which can be applied to construct functional connectivity matrix. One measure is phase coherence $(PC)$. Phase coherence $PC$ between two time-series $a$ and $b$ is defined as the following:

$$ PC_{ab} = \lvert  {R e^{i \Theta_{ab}}}  \rvert = \left| \frac{1}{T} \sum_{t=1}^{T} e^{i \theta_{ab}(t)} \right| ,  \\ $$

where $\theta_{ab}(t)$ is difference of phases of time-series $a$ and $b$ at time $t$:

$$ \theta_{ab}(t) = \theta_a(t) - \theta_b(t) \\ $$


• Construct a code for a function that computes $PC_{ij}$ for given time-series $i$ and $j$.

• Construct a code for a function that constructs $PC$ matrix which contain $PC_{ij}$ for all possible pairs of time_series.

• Use the codes to construct connectivity matrix as in Exercise 2.

• After the construction, we can proceed to apply the measures as in Exercise 3.



** EXERCISE 8: Pearson correlation coefficients (optional, possible for mini projects)**

• Another measure which can be used to construct connectivity matrix  is Pearson correlation coefficient $c$. Measure *Pearson* correlation coefficients ($c$) between all possible pairs, and contruct a correlation matrix with the coefficients as its element. The resulting matrix will be a symmetric matrix. The pearson correlation coefficient $c_{xy}$ between two data set  $x=\{x_1, x_2, x_3, ..., x_n \}$ and $y=\{y_1, y_2, y_3, ..., y_n \}$ is defined as the following: 

$$ c_{xy} = \frac {  \sum_{i=1}^{n} (x_i - \bar x) (y_i - \bar y) }  { \sqrt { \sum_{i=1}^{n} (x_i - \bar x )^2 }  \sqrt {\sum_{i=1}^{n} (y_i - \bar y)^2 }   } $$ 

where $\bar x$ and $\bar y$ are the mean of $x$ and $y$.

Alternatively, we can rewrite in the following way:

$$ c_{xy} = \frac { cov(x,y) } { \sqrt { var(x) \ var(y)  } } $$



where

$$ cov(x,y) = \langle  (x_i - \bar x)  (y_i - \bar y) \rangle _i \\
var(x,y) = \langle x_i - \bar x \rangle _i.$$

• You can construct a code for a function that computes $c_{ij}$ for given time-series $i$ and $j$, or you can use a numpy function, `np.corrcoef()`.

• Construct a code for a function that constructs correlation coefficient $c$ matrix which contain $c_{ij}$ for all possible pairs of time series.

• Use the codes to construct connectivity matrix as in Exercise 2.

• After the construction, we can proceed to Exercise 3. 